In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cd /kaggle

In [ ]:
ls

In [ ]:
pip install nltk


In [ ]:
pip install seqeval==0.0.3

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils import data
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.nn.utils.rnn import pad_sequence
from transformers import BertModel, BertTokenizer, BertForTokenClassification, AdamW
from seqeval.metrics import f1_score

In [ ]:
train_df = pd.read_csv("input/5046mk1/train.csv")

all_sentences = train_df["Sentence"]
labels = train_df["NER"]

train_df.head()

val_df = pd.read_csv("input/5046mk1/val.csv")
val_sentences = val_df["Sentence"]
val_labels = val_df["NER"]

In [ ]:
pair_data = list(zip(all_sentences, labels))
print(pair_data[0])

pair_val = list(zip(val_sentences, val_labels))
print(pair_val[4])

In [ ]:
split_pair_data = []
for sent, sent_label in pair_data:
    a_sent_pairs = []
    a_sent_words = sent.split(" ")
    a_sent_labels = sent_label.split(" ")
    
    temp = list(zip(a_sent_words, a_sent_labels))
    split_pair_data.append(temp)
    

split_pair_val = []
for sent, sent_label in pair_val:
    a_sent_pairs = []
    a_sent_words = sent.split(" ")
    a_sent_labels = sent_label.split(" ")
    
    temp = list(zip(a_sent_words, a_sent_labels))
    split_pair_val.append(temp)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:
processed_datapair = []
for pair in split_pair_data:
    tokenized_a_sentence_words = []
    labels_wordpiece = []
    
    for a_word, word_label in pair:     
        word_tokens = tokenizer.tokenize(a_word)
        label_nums = len(word_tokens)
        label_a_wordpiece = label_nums * [word_label]
        
        tokenized_a_sentence_words.extend(word_tokens)
        labels_wordpiece.extend(label_a_wordpiece)
    
    processed_datapair.append([tokenized_a_sentence_words, labels_wordpiece])
    

In [ ]:
processed_valpair = []
for pair in split_pair_val:
    tokenized_a_sentence_words = []
    labels_wordpiece = []
    
    for a_word, word_label in pair:     
        word_tokens = tokenizer.tokenize(a_word)
        label_nums = len(word_tokens)
        label_a_wordpiece = label_nums * [word_label]
        
        tokenized_a_sentence_words.extend(word_tokens)
        labels_wordpiece.extend(label_a_wordpiece)
    
    processed_valpair.append([tokenized_a_sentence_words, labels_wordpiece])

In [ ]:
all_labels = []
for pair in processed_datapair:
    label = pair[1]
    all_labels.append(label)
    
all_val_labels = []
for pair in processed_valpair:
    label = pair[1]
    all_val_labels.append(label)

In [ ]:
# print(len(all_labels))
# print(all_labels[0])

i=0
temp = []
for sent_label in all_labels:
#     print(sent_label)
    temp.extend(sent_label)
    
#     i+=1
#     if i == 3:
#         break

unique = list(set(temp))    
print(unique)

In [ ]:
labels = []
labels.append("[PAD]")
labels.extend(unique)
labels.append("[CLS]")
labels.append("[SEP]")

labels2idx = {labels:i for i, labels in enumerate(labels)}
labels2idx

In [ ]:
idx2label = {value : key  for key, value in labels2idx.items()}

idx2label


In [ ]:
tokens_sent = []
for pair in processed_datapair:
    tokens_sent.append(pair[0])

print(len(tokens_sent))
print(tokens_sent[1])

In [ ]:
tokens_sent_val = []
for pair in processed_valpair:
    tokens_sent_val.append(pair[0])

print(len(tokens_sent_val))
print(tokens_sent_val[1])

In [ ]:
input_ids = []
attention_masks = []

# For every sentence...
for sent in tokens_sent:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 75,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

In [ ]:
input_ids_val = []
attention_masks_val = []

# For every sentence...
for sent in tokens_sent_val:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 75,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_val.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_val.append(encoded_dict['attention_mask'])

In [ ]:
len(input_ids_val)
print(input_ids_val[0].size())
print(input_ids_val[5])

In [ ]:
labels_idx = []

for pair in processed_datapair:
    one_label_idx = []
    
    for label in pair[1]:
        one_label_idx.append(labels2idx[label])

    labels_idx.append(one_label_idx)


In [ ]:
print(labels_idx[0])


In [ ]:
labels_idx_val = []

for pair in processed_valpair:
    one_label_idx = []
    
    for label in pair[1]:
        one_label_idx.append(labels2idx[label])

    labels_idx_val.append(one_label_idx)


In [ ]:
max_len = 75
j = 0
label_pad = []

for label in labels_idx:
    label.insert(0, 6)
    label.append(7)
    if len(label) > max_len:
        temp = label[0:max_len]
        print("已经裁剪了"+str(j))
    else:
        diff = max_len - len(label)
        pad = diff*[0]
        temp = label
        temp.extend(pad)
    
    j += 1
    label_pad.append(temp)
        


In [ ]:
max_len = 75
j = 0
label_pad_val = []

for label in labels_idx_val:
    label.insert(0, 6)
    label.append(7)
    if len(label) > max_len:
        temp = label[0:max_len]
        print("已经裁剪了"+str(j))
    else:
        diff = max_len - len(label)
        pad = diff*[0]
        temp = label
        temp.extend(pad)
    
    j += 1
    label_pad_val.append(temp)

In [ ]:
# Wordpiece Label for NER
#https://github.com/google-research/bert/issues/560

class Nerdata(data.Dataset):
    def __init__(self, input_ids, attmask, labels):
        self.data = input_ids
        self.attmask = attmask
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data = self.data[idx]
        mask = self.attmask[idx]
        label = self.labels[idx]
        label = torch.tensor(label)
        
        return data, mask, label
        


In [ ]:
train_data = Nerdata(input_ids, attention_masks ,label_pad)

print(train_data.__len__())

val_data = Nerdata(input_ids_val, attention_masks_val, label_pad_val)
print(val_data.__len__())

print(type(label_pad))
print(label_pad[1])

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-large-cased', num_labels = 8, output_hidden_states = True)

In [ ]:
# dataloader_test = data.DataLoader(train_data, batch_size=2)

# for batches in tqdm(dataloader_test):
#     input_data = batches[0].view(-1,75)
#     mask = batches[1].view(-1,75)
#     label = batches[2]
#     loss, logits = model(input_data, token_type_ids=None,attention_mask=mask, labels=label)
#     break

In [ ]:
batch_size = 32
dataloader_train = data.DataLoader(train_data, batch_size=batch_size)
dataloader_val = data.DataLoader(val_data, batch_size=batch_size)

batch_num_val = len(dataloader_val)
batch_num_train = len(dataloader_train)
print(batch_num_val)
print(batch_num_train)

In [ ]:
total_epoch = 25

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=5e-5,
    eps=1e-8
)

In [ ]:
max_acc = 97

######################################  Train the model
model.cuda()
for epoch in range(total_epoch):
    train_acc_epoch = 0
    val_acc_epoch = 0
    predictions , true_labels = [], []
    predictions_val , true_labels_val = [], []

    for batches in tqdm(dataloader_train):
        
        input_data = batches[0].view(-1,75).cuda()
        mask = batches[1].view(-1,75).cuda()
        label = batches[2].cuda()
        
        model.train()
#         outputs = model(input_data, attention_masks = mask, labels=label)
        loss, logits, embeddings = model(input_data, token_type_ids=None,
                        attention_mask=mask, labels=label)

        loss_train =  loss
        loss_train.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(input_data, token_type_ids=None,
                        attention_mask=mask, labels=label)
            
        # Move logits and labels to CPU
        logits = outputs[1]
        y_hat = nn.functional.softmax(logits, dim=2)
        y_hat = torch.argmax(y_hat, dim=2)
   
        predictions.extend([list(p) for p in y_hat.cpu().detach().numpy()])
        true_labels.extend([list(q) for q in label.cpu().detach().numpy()])
        
    pred_tags = [idx2label[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if idx2label[l_i] != "[PAD]"]
    valid_tags = [idx2label[l_i] for l in true_labels
                                  for l_i in l if idx2label[l_i] != "[PAD]"]        
        
    print("train F1-Score: {0:.4}%".format(f1_score(pred_tags, valid_tags)*100))

    ##################################################################################################
    for batches in tqdm(dataloader_val):
        input_data = batches[0].view(-1,75).cuda()
        mask = batches[1].view(-1,75).cuda()
        label = batches[2].cuda()
        
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(input_data, token_type_ids=None,
                        attention_mask=mask, labels=label)
        # Move logits and labels to CPU

        logits = outputs[1]

        y_hat = nn.functional.softmax(logits, dim=2)
        y_hat = torch.argmax(y_hat, dim=2)
   
        predictions_val.extend([list(p) for p in y_hat.cpu().detach().numpy()])
        true_labels_val.extend([list(q) for q in label.cpu().detach().numpy()])
        
    pred_tags_val = [idx2label[p_i] for p, l in zip(predictions_val, true_labels_val)
                                 for p_i, l_i in zip(p, l) if idx2label[l_i] != "[PAD]"]
    valid_tags_val = [idx2label[l_i] for l in true_labels_val
                                  for l_i in l if idx2label[l_i] != "[PAD]"]        
        
    print("val F1-Score: {0:.4}%".format(f1_score(pred_tags_val, valid_tags_val)*100))
    
    if (f1_score(pred_tags_val, valid_tags_val)*100) > max_acc:
        torch.save(model, '/kaggle/working/best_model.' + str((f1_score(pred_tags_val, valid_tags_val)*100)) + 'pth')


# torch.save(model, "/kaggle/working/last"+ str(epoch+1)+ "_" + str((f1_score(pred_tags_val, valid_tags_val)*100)) + ".pth")

In [ ]:
type(embeddings)
len(embeddings[1])
embeddings[0].size()

In [ ]:
dataloader_embd = data.DataLoader(train_data, batch_size=1)

model_bert = torch.load('/kaggle/working/best_model.pth')

train_embds = []
for batches in tqdm(dataloader_embd):
    input_data = batches[0].view(-1,75).cuda()
    mask = batches[1].view(-1,75).cuda()
    label = batches[2].cuda()
        
    with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
        outputs = model(input_data, token_type_ids=None,
                        attention_mask=mask, labels=label)
        # Move logits and labels to CPU
    logits = outputs[2][0]
    
    train_embds.append(logits)



In [ ]:
dataloader_embd_val = data.DataLoader(val_data, batch_size=1)

model_bert = torch.load('/kaggle/working/best_model.pth')

val_embds = []
for batches in tqdm(dataloader_embd_val):
    input_data = batches[0].view(-1,75).cuda()
    mask = batches[1].view(-1,75).cuda()
    label = batches[2].cuda()
        
    with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
        outputs = model(input_data, token_type_ids=None,
                        attention_mask=mask, labels=label)
        # Move logits and labels to CPU
    logits = outputs[2][0]
    
    val_embds.append(logits)

In [ ]:
train_embds

val_embds